In [29]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [47]:
import requests
import pandas as pd
import datetime
import pytz
import schedule
import time
from datetime import datetime

In [48]:
url = "https://www.zse.co.zw/price-sheet/"

In [49]:
def get_todays_data():
  response = requests.get(url)
  data = pd.read_html(response.text)
  df = pd.DataFrame(data[0])
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.dropna()
  return df

def get_open_price():
  df = get_todays_data()
  open_price = df[['Company Name','Opening Price']].T
  open_price.columns  = open_price.iloc[0]
  open_price = open_price[1:]
  open_price.index.name = 'Date'
  open_price = open_price.rename(index={open_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return open_price

def get_close_price():
  df = get_todays_data()
  close_price = df[['Company Name','Closing Price']].T
  close_price.columns  = close_price.iloc[0]
  close_price = close_price[1:]
  close_price.index.name = 'Date'
  close_price = close_price.rename(index={close_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return close_price

def get_vol_traded():
  df = get_todays_data()
  vol_traded = df[['Company Name','Total Traded Volume']].T
  vol_traded.columns  = vol_traded.iloc[0]
  vol_traded = vol_traded[1:]
  vol_traded.index.name = 'Date'
  vol_traded = vol_traded.rename(index={vol_traded.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return vol_traded

In [50]:
global open_price
global close_price
global vol_traded
open_price = get_open_price()
close_price = get_close_price()
vol_traded = get_vol_traded()

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


In [62]:
#this is the code that needs to run everyday
def update_data():
  global open_price
  open_price = pd.concat([open_price, get_open_price()], axis=0) # Update the variables
  global close_price
  close_price = pd.concat([close_price, get_close_price()], axis=0)
  global vol_traded
  vol_traded = pd.concat([vol_traded, get_vol_traded()], axis=0)

  return open_price, close_price, vol_traded

update_data()

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


(Company Name                     Afdis Distillers Limited  \
 Date                                                        
 2024-10-16 15:53:42.704198+02:00                      700   
 2024-10-16 15:57:31.161371+02:00                      700   
 2024-10-16 15:58:11.427747+02:00                      700   
 
 Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
 Date                                                                                   
 2024-10-16 15:53:42.704198+02:00                                               55.5    
 2024-10-16 15:57:31.161371+02:00                                               55.5    
 2024-10-16 15:58:11.427747+02:00                                               55.5    
 
 Company Name                     Ariston Holdings Limited  \
 Date                                                        
 2024-10-16 15:53:42.704198+02:00                     7.96   
 2024-10-16 15:57:31.161371+02:00                     7

In [59]:
open_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-16 15:53:42.704198+02:00,700,55.5,7.96,1.5,2.64,7200,1400,1605,281.5,247.0524,...,2,165.1425,280,6.82,250,4.5,814.5714,0.0018,6,38
2024-10-16 15:57:31.161371+02:00,700,55.5,7.96,1.5,2.64,7200,1400,1605,281.5,247.0524,...,2,165.1425,280,6.82,250,4.5,814.5714,0.0018,6,38


In [60]:
close_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-16 15:53:46.714097+02:00,700,55.5,4.0091,1.5,2.64,7200,1400,1604.2308,281.5,247.0524,...,2,165.121,280,7,250,4.6815,814.5714,0.0018,6,38.5714
2024-10-16 15:57:35.241038+02:00,700,55.5,4.0091,1.5,2.64,7200,1400,1604.2308,281.5,247.0524,...,2,165.121,280,7,250,4.6815,814.5714,0.0018,6,38.5714


In [61]:
vol_traded

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-16 15:53:50.359222+02:00,0,0,953000,0,0,0,0,2600,0,0,...,117800,12400,0,400,0,230800,0,0,0,700
2024-10-16 15:57:43.127545+02:00,0,0,953000,0,0,0,0,2600,0,0,...,117800,12400,0,400,0,230800,0,0,0,700


In [46]:
#schedule.every(1).day.do(update_data)

#while True:
   #schedule.run_pending()
   #time.sleep(1)

   #print('jobs:', len(schedule.get_jobs()))

jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5
jobs: 5


/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-16 15:49:47.959099+02:00                      700   
2024-10-16 15:51:03.882520+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-16 15:49:47.959099+02:00                                               55.5    
2024-10-16 15:51:03.882520+02:00                                               55.5    

Company Name                     Ariston Holdings Limited  \
Date                                                        
2024-10-16 15:49:47.959099+02:00                     7.96   
2024-10-16 15:51:03.882520+02:00                     7.96   

Company Name                     Bridgefort Capital Limited  \
Date                                                          
2024-10-16 15:49:47.959099+02:0

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-16 15:49:47.959099+02:00                      700   
2024-10-16 15:51:03.882520+02:00                      700   
2024-10-16 15:51:16.522904+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-16 15:49:47.959099+02:00                                               55.5    
2024-10-16 15:51:03.882520+02:00                                               55.5    
2024-10-16 15:51:16.522904+02:00                                               55.5    

Company Name                     Ariston Holdings Limited  \
Date                                                        
2024-10-16 15:49:47.959099+02:00                     7.96   
2024-10-16 15:51:03.882520+02:00                     7.96   
2024-10-1

KeyboardInterrupt: 